In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pyscf
from pyscf import gto,scf,fci,lo,mcscf
from pyscf import ao2mo
from pyscf.tools import molden

In [ ]:
molname = 'hplane34'
natoms = 6
r = 1.0

mol = gto.Mole()
mol.verbose = 0

if molname == 'hchain':
    mol.atom = [['H',(0.0,0.0,i*r)] for i in range(natoms)]

elif molname == 'hplane':
    mol.atom = [['H',(0.,0.,0.)],
                ['H',(0.,0.,r)],
                ['H',(0.,0.,2*r)],
                ['H',(0.,r,0)],
                ['H',(0.,r,r)],
                ['H',(0.,r,2*r)]]

elif molname == 'hplane34':
    atom = []
    for i in range(3):
        for j in range(4):
            x = i * 4.0
            y = j * 4.0
            atom += [['H', (x, y, 0)]]
    mol.atom = atom
    mol.unit='bohr'
    
elif molname == 'hplane44':
    atom = []
    for i in range(4):
        for j in range(4):
            x = i * 4.0
            y = j * 4.0
            atom += [['H', (x, y, 0)]]
    mol.atom = atom
    mol.unit='bohr'

elif molname == 'n2':
    mol.atom = [['N',(0.,0.,0.)],
                ['N',(0.,0.,r)]]

elif molname == 'hf':
    mol.atom = [['H',(0.,0.,0.)],
                ['F',(0.,0.,r)]]

elif molname == 'c2':
    mol.atom = [['C',(0.,0.,0.)],
                ['C',(0.,0.,r)]]
elif molname == 'hcube332':
    atom = []
    for k in range(2):
        for j in range(3):
            for i in range(3):
                x = i * 4.0
                y = j * 4.0
                z = k * 4.0
                atom += [['H', (x, y, z)]]
                # atom += f"H, {x:.2f}, {y:.2f}, {z:.2f} ;\n"
    mol.atom = atom
    mol.unit='bohr'
    
mol.verbose = 5
mol.basis = 'sto-3g' 
mol.symmetry = True 
mol.charge = 0
mol.spin = 0
mol.build()

mf = scf.RHF(mol)
mf.init_guess = 'atom'
mf.level_shift = 0.0
mf.max_cycle = 100
mf.conv_tol=1.e-14
mf.scf()

In [ ]:
mf.mo_energy

In [ ]:
# from pyscf import mcscf, mrpt

# mc = mcscf.CASSCF(mf,4,4)
# ecas = mc.kernel()[0]
# ept2 = mrpt.NEVPT(mc).kernel()
# ecas, ecas + ept2

In [ ]:
# mc = mcscf.CASSCF(mf,12,12)
# ecas = mc.kernel()[0]
# ept2 = mrpt.NEVPT(mc).kernel()
# ecas, ecas + ept2

In [ ]:
# from pyscf import cc
# mycc = cc.CCSD(mf).run()
# print('CCSD total energy', mycc.e_tot)
# et = mycc.ccsd_t()
# print('CCSD(T) total energy', mycc.e_tot + et)

In [ ]:
mo = 'oao'
if mo == 'cmo':
    mo_coeff = mf.mo_coeff
elif mo == 'oao':
    mo_coeff = lo.orth_ao(mol, method="meta-lowdin")
molden.from_mo(mol, 'mo.molden', mo_coeff)

norb = mo_coeff.shape[1]
h1 = mo_coeff.T.dot(mf.get_hcore()).dot(mo_coeff)
eri = ao2mo.kernel(mol, mo_coeff)

In [ ]:
# #
# # FCI
# #

# cisolver = fci.direct_spin1.FCI(mol)
# cisolver = pyscf.fci.addons.fix_spin(cisolver,shift=0.05,ss=0)
# cisolver.nroots = 2 #10
# norb = mol.nao
# nelec = [mol.nelectron//2,mol.nelectron//2]
# e, ci = cisolver.kernel(h1, eri, norb, nelec, ecore=mol.energy_nuc(), max_cycle=1000)
# e

In [ ]:
eri = ao2mo.general(mol,(mo_coeff,mo_coeff,mo_coeff,mo_coeff),compact=0).reshape(norb,norb,norb,norb)

In [ ]:
kij = np.einsum('ijji->ij',eri)

# 1. generate a 1D ordering

In [ ]:
# import utils.graph.fielder
from utils.graph import fielder

forder = fielder.orbitalOrdering(kij,mode='kmat',debug=False)
forder

In [ ]:
from utils.graph import greedy

gorder0 = greedy.greedyOrdering(kij,iwt=0)[-1]
gorder1 = greedy.greedyOrdering(kij,iwt=1)[-1]

print(gorder0)
print(gorder1)

# 2. add connections to other sites based on weights on graph

In [ ]:
from utils.graph import nxutils

In [ ]:
graph0 = nxutils.fromKijToGraph(kij)
nxutils.displayGraph(graph0,kij)

In [ ]:
fgraph = nxutils.fromOrderToDiGraph(forder)
nxutils.displayCircular(fgraph)

In [ ]:
import networkx

networkx.__version__

In [ ]:
nxutils.displayGraphHighlight(graph0,kij,fgraph)

In [ ]:
maxdes = 2
fgraph2 = nxutils.addEdgesByGreedySearch(fgraph,kij,maxdes)
nxutils.displayCircular(fgraph2)

In [ ]:
nxutils.displayGraphHighlight(graph0,kij,fgraph2)

In [ ]:
maxdes = len(graph0.nodes)-1
graph2 = nxutils.addEdgesByGreedySearch(fgraph,kij,maxdes)
nxutils.displayCircular(graph2)

In [ ]:
nxutils.displayGraphHighlight(graph0,kij,graph2)

# greedy

In [ ]:
graph = nxutils.fromOrderToDiGraph(gorder0)
nxutils.displayCircular(graph)

In [ ]:
maxdes = 2
graph2 = nxutils.addEdgesByGreedySearch(graph,kij,maxdes)
nxutils.displayCircular(graph2)

# User defined graph

In [ ]:
import networkx as nx

# define the geomtry of lattice

edges = [[0,1],[1,2],[2,3],
                      [0,4],[1,5],[2,6],[3,7],
                      [4,5],[5,6],[6,7],
                      [4,8],[5,9],[6,10],[7,11],
                     [8,9],[9,10],[10,11],
                      [8,12],[9,13],[10,14],[11,15],
                     [12,13],[13,14],[14,15]
                     ]
tij = np.zeros((16,16))
for i,j in edges:
    tij[i,j] = 1
    tij[j,i] = 1

In [ ]:
graph = nx.Graph()
graph.add_edges_from(edges)
nnodes = len(graph.nodes)

In [ ]:
nx.draw_spring(graph, labels=dict(zip(range(nnodes),range(nnodes))))

In [ ]:
nxutils.displayCircular(graph)

In [ ]:
forder = fielder.orbitalOrdering(kij,'kmat')
print('forder=',forder)
fgraph = nxutils.fromOrderToDiGraph(forder)
nxutils.displayCircular(fgraph)

In [ ]:
maxdes = 1
fgraph2 = nxutils.addEdgesByGreedySearch(fgraph,kij,maxdes)
nxutils.displayCircular(fgraph2)

# Highlight path on graph

In [ ]:
pos = nx.spring_layout(graph, seed=0)
nx.draw_networkx_nodes(graph, pos, node_size=500, node_color='black')
labels = dict(zip(graph.nodes,graph.nodes))
nx.draw_networkx_labels(graph, pos, labels, font_size=15, font_color="whitesmoke")
nx.draw_networkx_edges(graph, pos, width=1.0, alpha=0.8)
nx.draw_networkx_edges(
    fgraph,
    pos,
    edgelist=list(fgraph.edges),
    edge_color="tab:red",
    width=5,
    alpha=0.6,
)
plt.show()

In [ ]:
pos = nx.spring_layout(graph, seed=0)
nx.draw_networkx_nodes(graph, pos, node_size=500, node_color='black')
labels = dict(zip(graph.nodes,graph.nodes))
nx.draw_networkx_labels(graph, pos, labels, font_size=15, font_color="whitesmoke")
nx.draw_networkx_edges(graph, pos, width=1.0, alpha=0.8)
nx.draw_networkx_edges(
    fgraph,
    pos,
    edgelist=list(fgraph.edges),
    edge_color="tab:red",
    width=5,
    alpha=0.8,
)
edgecomplement = [edge for edge in list(fgraph2.edges) if (edge[0],edge[1]) not in list(fgraph.edges) and (edge[1],edge[0]) not in list(fgraph.edges)]
print('edgecomplement=',edgecomplement)
nx.draw_networkx_edges(
    fgraph2,
    pos,
    edgelist=edgecomplement,
    edge_color="tab:green",
    width=5,
    alpha=0.5,
)
plt.show()